# Calculating entropy of lysozyme (Solute and Solvent)

## Loading Data
1. Load your data into a MDAnalysis Universe. 

In [1]:
import MDAnalysis as mda
# set the working dir to the root of repo inorder to use these path
tprfile = "data/1AKI_prod.tpr"
trrfile = "data/1AKI_prod.trr"
u = mda.Universe(tprfile, trrfile)

2. Trim the data to reduce analysis time

In [2]:
from CodeEntropy.IO import MDAUniverseHelper as MDAHelper
# since this trajectory only contains the DNA strand
selection_string_pre_process = 'protein'

reduced_frame = MDAHelper.new_U_select_frame(u)

reduced_atom = MDAHelper.new_U_select_atom(reduced_frame, selection_string_pre_process)



3. parse the data into a CodeEntropy data object

In [3]:
from CodeEntropy.ClassCollection import DataContainer as DC
dataContainer = DC.DataContainer(reduced_atom)

Number of atoms      : 1960
Number of frames     : 101


## Performing calculation
The total entropy for a system is taken as the sum of seven terms:
$S_{total} = S^{transvib}_{WM} + S^{rovib}_{WM} + S^{transvib}_{R} + S^{rovib}_{R} + S^{transvib}_{UA} + S^{rovib}_{UA} + S^{topo}_{UA}$

### Set Parameters

In [4]:
from CodeEntropy.FunctionCollection import EntropyFunctions as EF
#Only the part of interest remained
selection_string = "all"
# axis_list = ["C5'", "C4'", "C3'"] using default 
outfile = None
moutFile = None
nmdFile = None
csv_out = None
tScale = 1.0
fScale = 1.0
temper = 300.0 #K
verbose = 3
thread = 4

### Whole-molecule Level 
$S^{transvib}_{WM} + S^{rovib}_{WM}$

In [5]:
wm_entropyFF, wm_entropyTT = EF.compute_entropy_whole_molecule_level(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string, 
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose
)
print(f"wm_entropyFF = {wm_entropyFF}")
print(f"wm_entropyTT = {wm_entropyTT}")

------------------------------------------------------------
          Hierarchy level. --> Whole molecule <--           
------------------------------------------------------------
Total number of beads at the whole molecule level = 1
Assigning Translation and Rotation Axes @ whole molecule level-> Done
Updating Local coordinates-> Done
Updating Local forces-> Done
Updating Local torques-> Done
Weighting forces and torques-> Done
Updating the submatrices ... 
Finished updating submatrix FF
Finished updating submatrix TT
Done
Generating Quadrants-> Done
Diagonalizing-> Done
Changing the units of eigen values to SI units-> Done
Sorting spectrum in ascending order of frequencies-> Done
Entropy values:
FF Entropy (Whole mol level)             : 53.0696 J/mol/K
TT Entropy (Whole mol level)             : 48.4799 J/mol/K
wm_entropyFF = 53.069649409467544
wm_entropyTT = 48.47988928826726


### Residue level
$S^{transvib}_{R} + S^{rovib}_{R}$

In [6]:
res_entropyFF, res_entropyTT = EF.compute_entropy_residue_level(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string, 
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose,
)
# the result is Nan due to not enough sampling time see FAQ
print(f"res_entropyFF = {res_entropyFF}")
print(f"res_entropyTT = {res_entropyTT}")

------------------------------------------------------------
             Hierarchy level. --> Residues <--              
------------------------------------------------------------
LYS1
VAL2
PHE3
GLY4
ARG5
CYS6
GLU7
LEU8
ALA9
ALA10
ALA11
MET12
LYS13
ARG14
HIS15
GLY16
LEU17
ASP18
ASN19
TYR20
ARG21
GLY22
TYR23
SER24
LEU25
GLY26
ASN27
TRP28
VAL29
CYS30
ALA31
ALA32
LYS33
PHE34
GLU35
SER36
ASN37
PHE38
ASN39
THR40
GLN41
ALA42
THR43
ASN44
ARG45
ASN46
THR47
ASP48
GLY49
SER50
THR51
ASP52
TYR53
GLY54
ILE55
LEU56
GLN57
ILE58
ASN59
SER60
ARG61
TRP62
TRP63
CYS64
ASN65
ASP66
GLY67
ARG68
THR69
PRO70
GLY71
SER72
ARG73
ASN74
LEU75
CYS76
ASN77
ILE78
PRO79
CYS80
SER81
ALA82
LEU83
LEU84
SER85
SER86
ASP87
ILE88
THR89
ALA90
SER91
VAL92
ASN93
CYS94
ALA95
LYS96
LYS97
ILE98
VAL99
SER100
ASP101
GLY102
ASN103
GLY104
MET105
ASN106
ALA107
TRP108
VAL109
ALA110
TRP111
ARG112
ASN113
ARG114
CYS115
LYS116
GLY117
THR118
ASP119
VAL120
GLN121
ALA122
TRP123
ILE124
ARG125
GLY126
CYS127
ARG128
LEU129
Total number of beads 

/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/FunctionCollection/EntropyFunctions.py:47: RuntimeWarning: invalid value encountered in sqrt
  return nmp.sqrt((arg_lambdas)/UAC.get_KT2J(arg_temper))/(2*nmp.pi)
/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/FunctionCollection/EntropyFunctions.py:59: RuntimeWarning: invalid value encountered in sqrt
  afac = UAC.M2ANG * UAC.sqrtKG2AMU * nmp.divide(UAC.get_KT2J(arg_temper), nmp.sqrt(arg_lambdas))


### United Atom Level
$S^{transvib}_{UA} + S^{rovib}_{UA}$

In [7]:
UA_entropyFF, UA_entropyTT, res_df = EF.compute_entropy_UA_level_multiprocess(
    arg_hostDataContainer = dataContainer,
    arg_outFile = outfile,
    arg_selector = selection_string, 
    arg_moutFile = moutFile,
    arg_nmdFile = nmdFile,
    arg_fScale = fScale,
    arg_tScale = tScale,
    arg_temper = temper,
    arg_verbose = verbose,
    arg_csv_out= csv_out,
    arg_thread = thread
)
print(f"UA_entropyFF = {UA_entropyFF}")
print(f"UA_entropyTT = {UA_entropyTT}")
print("Per residue entropy:")
import pandas as pd
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(res_df)

Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (15,15) -> (7, 7)



Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (24,24) -> (10, 10)



Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (27,27) -> (20, 20)



Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (15,15) -> (7, 7)



Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (42,42) -> (19, 19)



Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (21,21) -> (12, 12)



Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (24,24) -> (13, 13)



Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (36,36) -> (17, 17)



Finished updating submatrix FF
Finished updating submatrix TT
A shape change has occured (21,21) -> (12, 12)

### Topographical entropy
$S^{topo}_{UA}$

In [8]:
result_topo_BB = EF.compute_topographical_entropy0_BB(
    arg_hostDataContainer = dataContainer, 
    arg_selector = selection_string,
    arg_outFile = outfile, 
    arg_verbose = verbose
)

print(f"result_topo_BB = {result_topo_BB}")

------------------------------------------------------------
Topographical entropy of BB dihedrals 
 computed using the pLogp formalism
------------------------------------------------------------
Found a total of 886 BB dihedrals.
Dihedral 143  145  147  153   : -0.0000 (   10)
Dihedral 1136 1151 1153 1155  : -0.0000 (-10000)
Dihedral 650  660  662  664   : -0.0000 (-10000)
Dihedral 1312 1311 1313 1315  : -0.0000 (-10000)
Dihedral 1397 1399 1405 1407  : 2.0939 (   94)
Dihedral 67   69   89   91    : 0.4618 (    5)
Dihedral 1073 1064 1065 1076  : -0.0000 (   70)
Dihedral 1824 1832 1830 1831  : -0.0000 (-10000)
Dihedral 1009 1017 1019 1021  : -0.0000 (-10000)
Dihedral 153  155  157  159   : -0.0000 (   11)
Dihedral 383  398  400  402   : -0.0000 (-10000)
Dihedral 1064 1065 1076 1078  : 2.4983 (   70)
Dihedral 537  535  546  548   : 5.7301 (   35)
Dihedral 636  638  640  646   : 0.8087 (   42)
Dihedral 758  760  770  772   : -0.0000 (   51)
Dihedral 1369 1381 1383 1385  : -0.0000 (-10000

In [9]:
result_topo_SC = EF.compute_topographical_entropy0_SC(
    arg_hostDataContainer = dataContainer, 
    arg_selector = selection_string,
    arg_outFile = outfile, 
    arg_verbose = verbose
)

print(f"result_topo_SC = {result_topo_SC}")

------------------------------------------------------------
Topographical entropy of residue side chains 
 computed using all the dihedrals with pLogp formalism
------------------------------------------------------------
----------Working on resid : 1 (LYS)----------
Found 5 exclusive dihedrals in residue LYS
Dihedral 4    6    9    12    : -0.0000
Dihedral 9    12   15   18    : 0.8087
Dihedral 9    6    4    22    : 5.7137
Dihedral 6    9    12   15    : 5.6448
Dihedral 0    4    6    9     : 5.7301
Side Chain Topographical Entropy (LYS 1) : 17.8974
----------Working on resid : 2 (VAL)----------
Found 4 exclusive dihedrals in residue VAL
Dihedral 34   28   26   38    : 0.4618
Dihedral 24   26   28   34    : 0.4618
Dihedral 24   26   28   30    : -0.0000
Dihedral 30   28   26   38    : -0.0000
Side Chain Topographical Entropy (VAL 2) : 0.9237
----------Working on resid : 3 (PHE)----------
Found 11 exclusive dihedrals in residue PHE
Dihedral 47   48   52   56    : -0.0000
Dihedral 48

### Total Entropy

In [10]:
total = wm_entropyFF + wm_entropyTT + UA_entropyFF + UA_entropyTT + result_topo_SC +result_topo_BB
print(f"Total Entropy = {total} J/mol/K")

Total Entropy = 6897.588318899479 J/mol/K


## Load data into POSEIDON object

In [11]:
from CodeEntropy.ClassCollection.PoseidonClass import Poseidon
poseidon_object = Poseidon(container=u, start=0, end=20)

2022-07-01 07:54:39.259567
frame = 0
< Timestep 0 with unit cell dimensions [69.57307 69.57307 69.57307 90.      90.      90.     ] >
0:00:23.497173
frame = 1
< Timestep 1 with unit cell dimensions [69.476456 69.476456 69.476456 90.       90.       90.      ] >


/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/poseidon/extractData/HBRAD.py:192: RuntimeWarning: invalid value encountered in arccos
  angle1 = np.arccos(cosine_angle)


0:00:43.408170
frame = 2
< Timestep 2 with unit cell dimensions [69.61369 69.61369 69.61369 90.      90.      90.     ] >
0:01:02.702662
frame = 3
< Timestep 3 with unit cell dimensions [69.52697 69.52697 69.52697 90.      90.      90.     ] >
0:01:21.791872
frame = 4
< Timestep 4 with unit cell dimensions [69.57063 69.57063 69.57063 90.      90.      90.     ] >
0:01:42.639399
frame = 5
< Timestep 5 with unit cell dimensions [69.51649 69.51649 69.51649 90.      90.      90.     ] >
0:02:03.379333
frame = 6
< Timestep 6 with unit cell dimensions [69.44309 69.44309 69.44309 90.      90.      90.     ] >
0:02:23.789225
frame = 7
< Timestep 7 with unit cell dimensions [69.6301 69.6301 69.6301 90.     90.     90.    ] >
0:02:44.362540
frame = 8
< Timestep 8 with unit cell dimensions [69.58307 69.58307 69.58307 90.      90.      90.     ] >
0:03:03.665345
frame = 9
< Timestep 9 with unit cell dimensions [69.59987 69.59987 69.59987 90.      90.      90.     ] >
0:03:22.878227
frame = 10
< Ti

## Calculate Entropy

### Whole Molecule level

In [12]:
result_wm = poseidon_object.run_analysis(level_list = ['moleculeLevel'], verbose=False, forceUnits="Kcal") # this is because the forces value supplied in this trajectory is in Kcal
print(result_wm)

2022-07-01 08:01:24.918433

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 1.465 GB
0:00:00.000895
memory use: 1.444 GB
0:00:15.993168
0:00:15.993540
memory use: 1.444 GB

Total number of frames: 20.0
Number of atoms in each frame: 11645
Number of variables in each list: 19

2. Process Dictionaries
['moleculeLevel']
---level: moleculeLevel


0:00:18.758795
{'moleculeLevel': {'solventData':    nearest assigned shell_num   variable      value count
0      ALA      SOL       1.0  Sor_test2   2.045613   270
1      ALA      SOL       1.0     Strans  10.853957   270
2      ALA      SOL       1.0       Srot   0.541862   270
3      ALA      SOL       1.0      count       13.5   270
4      ARG      SOL       1.0  Sor_test2   4.307759  2145
..     ...      ...       ...        ...        ...   ...
79     TYR      SOL       1.0      count       8.45   169
80     VAL      SOL       1.

/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/poseidon/analysis/EECalculation.py:859: RuntimeWarning: invalid value encountered in double_scalars
  w = w ** 0.5


### Residue Level

In [13]:
result_res = poseidon_object.run_analysis(level_list = ['residLevel_resname'], verbose=False, forceUnits="Kcal") # this is because the forces value supplied in this trajectory is in Kcal
print(result_res)

2022-07-01 08:01:43.702703

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 1.442 GB
0:00:00.001196
memory use: 1.442 GB
0:00:16.582369
0:00:16.582742
memory use: 1.442 GB

Total number of frames: 20.0
Number of atoms in each frame: 11645
Number of variables in each list: 19

2. Process Dictionaries
['residLevel_resname']
---level: residLevel_resname


/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/poseidon/analysis/EECalculation.py:859: RuntimeWarning: invalid value encountered in double_scalars
  w = w ** 0.5




0:00:22.815625
{'residLevel_resname': {'solventData':      nearest assigned shell_num   variable      value count
0     ALA_10      SOL       1.0  Sor_test2   2.050395    19
1     ALA_10      SOL       1.0     Strans   8.909477    19
2     ALA_10      SOL       1.0       Srot   0.379505    19
3     ALA_10      SOL       1.0      count       0.95    19
4    ALA_107      SOL       1.0  Sor_test2   1.684203    60
..       ...      ...       ...        ...        ...   ...
515    VAL_2      SOL       1.0      count       6.75   135
516   VAL_99      SOL       1.0  Sor_test2        0.0    17
517   VAL_99      SOL       1.0     Strans  14.724812    17
518   VAL_99      SOL       1.0       Srot   1.644911    17
519   VAL_99      SOL       1.0      count       0.85    17

[520 rows x 6 columns], 'soluteData':      resName      variable      value count
0     ALA_10     WM_Strans   1.844334    20
1     ALA_10       WM_Srot   2.242724    20
2     ALA_10  WM_UA_Strans   0.018059    20
3     ALA

### United Atom Level

In [14]:
result_res = poseidon_object.run_analysis(level_list = ['residLevel_resname'], verbose=False, forceUnits="Kcal") # this is because the forces value supplied in this trajectory is in Kcal
print(result_res)

2022-07-01 08:02:06.562839

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 1.442 GB
0:00:00.001819
memory use: 1.442 GB
0:00:17.453313
0:00:17.454214
memory use: 1.442 GB

Total number of frames: 20.0
Number of atoms in each frame: 11645
Number of variables in each list: 19

2. Process Dictionaries
['residLevel_resname']
---level: residLevel_resname


/home/donald/miniconda3/envs/CEtrial/lib/python3.10/site-packages/CodeEntropy/poseidon/analysis/EECalculation.py:859: RuntimeWarning: invalid value encountered in double_scalars
  w = w ** 0.5




0:00:23.437742
{'residLevel_resname': {'solventData':      nearest assigned shell_num   variable      value count
0     ALA_10      SOL       1.0  Sor_test2   2.050395    19
1     ALA_10      SOL       1.0     Strans   8.909477    19
2     ALA_10      SOL       1.0       Srot   0.379505    19
3     ALA_10      SOL       1.0      count       0.95    19
4    ALA_107      SOL       1.0  Sor_test2   1.684203    60
..       ...      ...       ...        ...        ...   ...
515    VAL_2      SOL       1.0      count       6.75   135
516   VAL_99      SOL       1.0  Sor_test2        0.0    17
517   VAL_99      SOL       1.0     Strans  14.724812    17
518   VAL_99      SOL       1.0       Srot   1.644911    17
519   VAL_99      SOL       1.0      count       0.85    17

[520 rows x 6 columns], 'soluteData':      resName      variable      value count
0     ALA_10     WM_Strans   1.844334    20
1     ALA_10       WM_Srot   2.242724    20
2     ALA_10  WM_UA_Strans   0.018059    20
3     ALA

### Solute Contact

In [15]:
result_solcon = poseidon_object.run_analysis(level_list = ['soluteContacts'], verbose=False, forceUnits="Kcal") # this is because the forces value supplied in this trajectory is in Kcal
print(result_solcon)

2022-07-01 08:02:30.038599

solvent: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

water: ['WAT', 'wat', 'SOL', 'H2O', 'h2o', 'WAT_O', 'TIP3']

1. Populate Dictionaries

memory use: 1.442 GB
0:00:00.002518
memory use: 1.442 GB
0:00:03.103853
0:00:03.104352
memory use: 1.442 GB

Total number of frames: 20.0
Number of atoms in each frame: 11645
Number of variables in each list: 19

2. Process Dictionaries
['soluteContacts']
---level: soluteContacts


0:00:14.487437
{'soluteContacts': {'solventData':               nearest assigned shell_num   variable      value count
0     ALA_107_ALA_107      SOL       1.0  Sor_test2   0.483728    13
1     ALA_107_ALA_107      SOL       1.0     Strans  12.249477    13
2     ALA_107_ALA_107      SOL       1.0       Srot   1.600586    13
3     ALA_107_ALA_107      SOL       1.0      count       0.65    13
4     ALA_107_ARG_112      SOL       1.0  Sor_test2   0.173337    18
...               ...      ...       ...        ...        ...   ...
5691  